In [4]:
import random
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

In [5]:
df = pd.read_csv('GLIF dataset (1).csv', low_memory=False)
display(df['severity'].value_counts())
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

severity
medium          5450
small           2586
unknown          827
large            654
...              357
very_large       101
Medium            72
Unknown           19
Large             17
Small             10
catastrophic       4
landslide          2
Very...large       1
Very_large         1
Name: count, dtype: int64

In [6]:

X = df.copy()
y = X.landslide
columns=[]
for i in range(9, 2, -1):
    columns.append('humidity' + str(i))
    columns.append('ARI' + str(i))
    # columns.append('wind' + str(i))
X = X[columns]
for i in range(9, 2, -1):
    X['ARI' + str(i)] = X['ARI' + str(i)].apply(float)
X



,humidity9,ARI9,humidity8,ARI8,humidity7,ARI7,humidity6,ARI6,humidity5,ARI5,humidity4,ARI4,humidity3,ARI3
0,83,3.168936,81,1.951982,87,1.881104,86,0.506835,80,0.161354,69,0.096520,98,7.694154
1,96,0.000000,98,0.000000,97,0.000000,99,0.000000,100,0.000000,97,0.000000,96,0.000000
2,81,3.899664,82,13.202904,83,9.537894,83,6.238562,94,7.603534,97,5.412976,98,11.647651
3,88,3.213674,68,1.920537,92,5.646332,95,44.401469,92,21.932705,87,9.438918,92,23.445983
4,58,0.606140,87,0.380326,97,6.140801,98,15.191881,97,6.609132,92,7.095303,91,3.371466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20103,55,0.455124,39,0.268646,41,0.177701,86,7.463486,92,44.555368,82,25.041942,76,22.281705
20104,83,4.037510,83,2.758766,84,10.426535,88,55.569273,89,171.916317,87,54.238654,85,43.443317
20105,98,5.346890,98,6.276276,98,7.951696,98,2.849562,97,3.410040,98,7.586011,97,2.148039
20106,96,0.714787,91,0.338082,95,11.378107,98,8.904781,99,7.035446,82,2.794186,71,1.393536


In [7]:
columns, len(columns)

(['humidity9',
  'ARI9',
  'humidity8',
  'ARI8',
  'humidity7',
  'ARI7',
  'humidity6',
  'ARI6',
  'humidity5',
  'ARI5',
  'humidity4',
  'ARI4',
  'humidity3',
  'ARI3'],
 14)

In [8]:

y.value_counts()


landslide
1    10101
0    10007
Name: count, dtype: int64

In [9]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train[0]


array([45.        ,  0.        , 56.        ,  0.        , 93.        ,
       12.10479937, 97.        ,  3.22463918, 90.        ,  3.97429888,
       95.        ,  3.93709151, 95.        ,  1.41162025])

In [10]:
len(X_test), len(y_test)

(4022, 4022)

In [11]:
len(X_train)

16086

In [12]:
X_train[0].shape

(14,)

In [13]:
y_test.value_counts()

landslide
0    2012
1    2010
Name: count, dtype: int64

In [14]:
y_train = np.array(y_train)
y_test = np.array(y_test)
# len(X_train) # of batches
# 5 timesteps (days 7 - 3)
# 2 features per timestep
X_train=X_train.reshape((len(X_train), 7, 2))
X_test=X_test.reshape((len(X_test), 7, 2))

print(X_train.shape)
X_train[0]


(16086, 7, 2)


array([[45.        ,  0.        ],
       [56.        ,  0.        ],
       [93.        , 12.10479937],
       [97.        ,  3.22463918],
       [90.        ,  3.97429888],
       [95.        ,  3.93709151],
       [95.        ,  1.41162025]])

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, BatchNormalization, LeakyReLU

model = Sequential()

# 1st Bidirectional LSTM Layer (256 units)
model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=(7, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# 2nd Bidirectional LSTM Layer (128 units)
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# 3rd Bidirectional LSTM Layer (64 units)
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# 4th Bidirectional LSTM Layer (32 units)
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.25))

# Dense Layers
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dropout(0.3))

model.add(Dense(128))
model.add(LeakyReLU())
model.add(Dropout(0.3))

model.add(Dense(64))
model.add(LeakyReLU())
model.add(Dropout(0.2))

model.add(Dense(32))
model.add(LeakyReLU())
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

# Train
model.fit(X_train, y_train, epochs=25, batch_size=64, verbose=2)
X_train = np.array(X_train)
X_test = np.array(X_test)

# len(X_train) # of batches
# 5 timesteps (days 7 - 3)
# 2 features per timestep
X_train=X_train.reshape((len(X_train), 7, 2))
X_test=X_test.reshape((len(X_test), 7, 2))

print(X_train.shape)
X_train[0]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
252/252 - 19s - 77ms/step - accuracy: 0.6683 - loss: 0.6165
Epoch 2/25
252/252 - 13s - 50ms/step - accuracy: 0.6846 - loss: 0.6044
Epoch 3/25
252/252 - 14s - 57ms/step - accuracy: 0.6886 - loss: 0.5959
Epoch 4/25
252/252 - 14s - 56ms/step - accuracy: 0.6987 - loss: 0.5898
Epoch 5/25
252/252 - 15s - 60ms/step - accuracy: 0.7034 - loss: 0.5825
Epoch 6/25
252/252 - 15s - 58ms/step - accuracy: 0.7102 - loss: 0.5805
Epoch 7/25
252/252 - 16s - 65ms/step - accuracy: 0.7125 - loss: 0.5766
Epoch 8/25
252/252 - 18s - 71ms/step - accuracy: 0.7176 - loss: 0.5725
Epoch 9/25
252/252 - 15s - 60ms/step - accuracy: 0.7189 - loss: 0.5681
Epoch 10/25
252/252 - 26s - 102ms/step - accuracy: 0.7175 - loss: 0.5663
Epoch 11/25
252/252 - 15s - 59ms/step - accuracy: 0.7229 - loss: 0.5640
Epoch 12/25
252/252 - 14s - 57ms/step - accuracy: 0.7198 - loss: 0.5611
Epoch 13/25
252/252 - 14s - 55ms/step - accuracy: 0.7242 - loss: 0.5580
Epoch 14/25
252/252 - 14s - 56ms/step - accuracy: 0.7296 - loss: 0.5534


array([[45.        ,  0.        ],
       [56.        ,  0.        ],
       [93.        , 12.10479937],
       [97.        ,  3.22463918],
       [90.        ,  3.97429888],
       [95.        ,  3.93709151],
       [95.        ,  1.41162025]])

In [16]:
model.evaluate(X_test, y_test, return_dict=True)

126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7410 - loss: 0.5258


{'accuracy': 0.7337145805358887, 'loss': 0.5379607677459717}

In [17]:
# Calculate precision, recall, F1 score, and confusion matrix
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

# Get predictions (probabilities)
y_pred_prob = model.predict(X_test)

# Convert probabilities to binary predictions using 0.5 threshold
y_pred = (y_pred_prob > 0.5).astype(int)

# Print detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Calculate individual metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("\nIndividual Metrics:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# Display confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)
print("\nFormat: [[TN, FP], [FN, TP]]")


126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.80      0.75      2012
           1       0.77      0.67      0.71      2010

    accuracy                           0.73      4022
   macro avg       0.74      0.73      0.73      4022
weighted avg       0.74      0.73      0.73      4022


Individual Metrics:
Precision: 0.7700
Recall: 0.6662
F1 Score: 0.7143
Accuracy: 0.7337

Confusion Matrix:
[[1612  400]
 [ 671 1339]]

Format: [[TN, FP], [FN, TP]]
